In [21]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from sklearn.preprocessing import LabelEncoder

# Define project Path in Colab
PROJECT_BASE_PATH = '/content/drive/MyDrive/Project_01' 

# ADD 'src' DIRECTORY TO PYTHON PATH
SRC_PATH = os.path.join(PROJECT_BASE_PATH, 'src')

# verify if SRC_PATH is already in sys.path
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print("✅ Successfully added 'src' directory to Python path.")

# IMPORT Paths CLASS FROM config MODULE
from config import Paths

In [25]:
try:
    from config import Paths
    
    # 3. Inicialize a instância com um nome único (cfg)
    cfg = Paths(PROJECT_BASE_PATH) # <-- Mudança aqui
    cfg.create_dirs() 
    
    print("\n✅ Project configuration (Paths) initialized successfully.")
    print(f"Raw Data Path check: {cfg.TRAIN_RAW_FILE}")
    
except ImportError:
    print("❌ Error: Could not import Paths from config module.")


✅ Project configuration (Paths) initialized successfully.
Raw Data Path check: /content/drive/MyDrive/Project_01/data/raw/application_train.csv


In [ ]:
DATA_DIR_PROCESSED = os.path.join(PROJECT_BASE_PATH, 'data', 'processed')
TRAIN_PROCESSED_FILE = os.path.join(DATA_DIR_PROCESSED, 'train_enriched.csv')
TEST_PROCESSED_FILE = os.path.join(DATA_DIR_PROCESSED, 'test_enriched.csv')

In [ ]:
try:
    # --- LOAD TRAIN DATA USING CONFIG PATHS ---
    df_train_final = pd.read_csv(cfg.TRAIN_PROCESSED_FILE)
    
    # ESSENTIAL FIX: Ensures TIME_INDEX is of temporal type after loading from CSV
    df_train_final['TIME_INDEX'] = pd.to_datetime(df_train_final['TIME_INDEX'])
    
    print(f"✅ Loaded final training data. Shape: {df_train_final.shape}")

    # --- LOAD TEST DATA USING CONFIG PATHS ---
    df_test_final = pd.read_csv(cfg.TEST_PROCESSED_FILE)
    
    # Apply the same temporal conversion to the Test Set
    df_test_final['TIME_INDEX'] = pd.to_datetime(df_test_final['TIME_INDEX'])
    
    print(f"✅ Loaded final testing data. Shape: {df_test_final.shape}")
except FileNotFoundError:
    print(f"❌ ERROR: Final processed file not found. Check if Block 9 was run and files exist at: {path.DATA_PROCESSED_DIR}")
    df_train_final = None
    df_test_final = None
if df_train_final is not None and df_test_final is not None:
    print("--- Starting Feature Engineering ---")
    
    # --- 1. Create Lagged and Dynamic Macro Features (TRAIN SET) ---
    
    # KeyError/SetIndex Fix: Ensures TIME_INDEX is a regular column before setting
    if 'TIME_INDEX' not in df_train_final.columns and 'TIME_INDEX' in df_train_final.index.names:
        df_train_final = df_train_final.reset_index(level='TIME_INDEX')
        print("💡 TRAIN: TIME_INDEX restored from index to column.")
    
    # Temporarily set TIME_INDEX as the index for time-series operations
    df_train_final = df_train_final.set_index('TIME_INDEX')
    
    # Use the correct column names
    macro_features_to_engineer = ['SELIC', 'IPCA'] 
    
    for col in macro_features_to_engineer:
        # A) Lag (Previous Month's Value)
        df_train_final[f'{col}_LAG1'] = df_train_final.groupby('SK_ID_CURR')[col].shift(1)

        # B) Change (Current Month - Previous Month)
        df_train_final[f'{col}_CHANGE'] = df_train_final[col] - df_train_final[f'{col}_LAG1']
        
        # C) Trend (3-Month Rolling Mean)
        df_train_final[f'{col}_ROLLING_MEAN3'] = df_train_final.groupby('SK_ID_CURR')[col].transform(
            lambda x: x.rolling(window=3, min_periods=1).mean().shift(1)
        )
        
    print(f"✅ TRAIN: Created dynamic features: SELIC/IPCA LAGs, Changes, and Rolling Means.")

    # --- 2. Temporal Features from TIME_INDEX (TRAIN SET) ---
    
    # Extract month and year using the already defined index
    df_train_final['MONTH_OF_YEAR'] = df_train_final.index.to_series().dt.month
    df_train_final['YEAR'] = df_train_final.index.to_series().dt.year
    
    print("✅ TRAIN: Created cyclical temporal features (MONTH_OF_YEAR, YEAR).")

    # Restore TIME_INDEX as a regular column
    df_train_final = df_train_final.reset_index()

    print(f"\nTraining set shape after Macro Feature Engineering: {df_train_final.shape}")
    
    
    # --------------------------------------------------------------------------------
    # --- REPEAT FOR TEST SET ---
    # --------------------------------------------------------------------------------
    
    # KeyError/SetIndex Fix for the Test Set
    if 'TIME_INDEX' not in df_test_final.columns and 'TIME_INDEX' in df_test_final.index.names:
        df_test_final = df_test_final.reset_index(level='TIME_INDEX')
        print("💡 TEST: TIME_INDEX restored from index to column.")
    
    # Temporarily set TIME_INDEX as the index for time-series operations
    df_test_final = df_test_final.set_index('TIME_INDEX')
    
    for col in macro_features_to_engineer:
        df_test_final[f'{col}_LAG1'] = df_test_final.groupby('SK_ID_CURR')[col].shift(1)
        df_test_final[f'{col}_CHANGE'] = df_test_final[col] - df_test_final[f'{col}_LAG1']
        df_test_final[f'{col}_ROLLING_MEAN3'] = df_test_final.groupby('SK_ID_CURR')[col].transform(
            lambda x: x.rolling(window=3, min_periods=1).mean().shift(1)
        )
        
    print(f"✅ TEST: Created dynamic features: SELIC/IPCA LAGs, Changes, and Rolling Means.")

    # Temporal Features (TEST SET)
    df_test_final['MONTH_OF_YEAR'] = df_test_final.index.to_series().dt.month
    df_test_final['YEAR'] = df_test_final.index.to_series().dt.year
    
    print("✅ TEST: Created cyclical temporal features (MONTH_OF_YEAR, YEAR).")

    # Restore TIME_INDEX as a regular column
    df_test_final = df_test_final.reset_index()

    print(f"\nTesting set shape after Macro Feature Engineering: {df_test_final.shape}")

✅ Loaded final training data. Shape: (307511, 125)
✅ Loaded final testing data. Shape: (48744, 124)
--- Starting Feature Engineering ---
✅ TRAIN: Created dynamic features: SELIC/IPCA LAGs, Changes, and Rolling Means.
✅ TRAIN: Created cyclical temporal features (MONTH_OF_YEAR, YEAR).

Training set shape after Macro Feature Engineering: (307511, 133)
✅ TEST: Created dynamic features: SELIC/IPCA LAGs, Changes, and Rolling Means.
✅ TEST: Created cyclical temporal features (MONTH_OF_YEAR, YEAR).

Testing set shape after Macro Feature Engineering: (48744, 132)


In [30]:
print("--- Starting Micro Feature Engineering ---")

# Treat DAYS_EMPLOYED anomaly (365243 days ≈ 1000 years) as missing (NaN) and flag it

#------------------------------------------------------------
# --- IMPORTANT DATA CLEANING NOTE: DAYS_EMPLOYED ANOMALY ---
# The value 365243 in DAYS_EMPLOYED is a known data anomaly specific to the Home Credit dataset.
# It represents approximately 1000 years, and is used by the client/bank to code for
# applicants who are currently **unemployed** or whose employment status is unverified.
#
# Best Practice Treatment:
# 1. Replace the anomalous value (365243) with **NaN** to treat it as a missing value.
# 2. Create a new **binary feature** (DAYS_EMPLOYED_ANOM) to allow the model (like LightGBM)
#    to explicitly learn the predictive power of this specific 'unemployed/anomaly' group.
# This prevents the model from interpreting 365243 as a literal, extremely long employment history.
# -----------------------------------------------------------
# --- 1. Anomaly Treatment and Transformation ---

DAYS_EMPLOYED_ANOMALY_VALUE = 365243

# Treat DAYS_EMPLOYED anomaly:
# 1. Flag the anomaly first
df_train_final['DAYS_EMPLOYED_ANOM'] = df_train_final['DAYS_EMPLOYED'] == DAYS_EMPLOYED_ANOMALY_VALUE
df_test_final['DAYS_EMPLOYED_ANOM'] = df_test_final['DAYS_EMPLOYED'] == DAYS_EMPLOYED_ANOMALY_VALUE

# 2. Replace the anomalous value with NaN using direct assignment (CORRIGIDO)
# Remove inplace=True and assign back to the column name
df_train_final['DAYS_EMPLOYED'] = df_train_final['DAYS_EMPLOYED'].replace({DAYS_EMPLOYED_ANOMALY_VALUE: np.nan})
df_test_final['DAYS_EMPLOYED'] = df_test_final['DAYS_EMPLOYED'].replace({DAYS_EMPLOYED_ANOMALY_VALUE: np.nan})

print("✅ Anomalous DAYS_EMPLOYED treated and flagged.")

# Transform DAYS_BIRTH into AGE_YEARS
df_train_final['AGE_YEARS'] = df_train_final['DAYS_BIRTH'] / -365
df_test_final['AGE_YEARS'] = df_test_final['DAYS_BIRTH'] / -365

# --- 2. Ratio Feature Creation ---

df_train_final['INCOME_PER_AGE'] = df_train_final['AMT_INCOME_TOTAL'] / df_train_final['AGE_YEARS']
df_test_final['INCOME_PER_AGE'] = df_test_final['AMT_INCOME_TOTAL'] / df_test_final['AGE_YEARS']

df_train_final['CREDIT_INCOME_RATIO'] = df_train_final['AMT_CREDIT'] / df_train_final['AMT_INCOME_TOTAL']
df_test_final['CREDIT_INCOME_RATIO'] = df_test_final['AMT_CREDIT'] / df_test_final['AMT_INCOME_TOTAL']

df_train_final['ANNUITY_CREDIT_RATIO'] = df_train_final['AMT_ANNUITY'] / df_train_final['AMT_CREDIT']
df_test_final['ANNUITY_CREDIT_RATIO'] = df_test_final['AMT_ANNUITY'] / df_test_final['AMT_CREDIT']

df_train_final['ANNUITY_INCOME_RATIO'] = df_train_final['AMT_ANNUITY'] / df_train_final['AMT_INCOME_TOTAL']
df_test_final['ANNUITY_INCOME_RATIO'] = df_test_final['AMT_ANNUITY'] / df_test_final['AMT_INCOME_TOTAL']

print("✅ Ratio and new temporal features created.")
print(f"Training set shape after Micro Feature Engineering: {df_train_final.shape}")


--- Starting Micro Feature Engineering ---
✅ Anomalous DAYS_EMPLOYED treated and flagged.
✅ Ratio and new temporal features created.
Training set shape after Micro Feature Engineering: (307511, 139)


In [31]:
from sklearn.preprocessing import LabelEncoder

print("--- Starting Feature Encoding (Block 13) ---")

# --- 1. Identify Categorical Columns ---
# Select columns that are still of type 'object'
object_cols_train = df_train_final.select_dtypes(include=['object']).columns.tolist()

# --- 2. Label Encoding (Binary Columns: 2 Unique Categories) ---

# Find columns with exactly two unique categories
le_cols = [col for col in object_cols_train if df_train_final[col].nunique() == 2]
print(f"Applying Label Encoding to {len(le_cols)} features.")

for col in le_cols:
    le = LabelEncoder()
    
    # Fit on the training set and transform both
    df_train_final[col] = le.fit_transform(df_train_final[col])
    
    # Transform the test set using the same fitted encoder
    df_test_final[col] = le.transform(df_test_final[col])

print("✅ Label Encoding completed.")


# --- 3. One-Hot Encoding (Multi-Category Columns: 3+ Categories) ---

# Remaining columns (those not label encoded) need One-Hot Encoding
ohe_cols = [col for col in object_cols_train if col not in le_cols]
print(f"Applying One-Hot Encoding to {len(ohe_cols)} features.")

# Concatenate for safe One-Hot Encoding to ensure same number of columns in both sets
# Drop TARGET before concatenation
df_target = df_train_final['TARGET'] # Save the TARGET column
df_combined = pd.concat([df_train_final.drop('TARGET', axis=1), df_test_final], axis=0)

# Apply get_dummies to the combined dataset
df_combined = pd.get_dummies(df_combined, columns=ohe_cols, dummy_na=False)

# Re-separate the train and test sets
df_train_final = df_combined.iloc[:len(df_train_final), :].copy()
df_test_final = df_combined.iloc[len(df_train_final):, :].copy()

# Add the TARGET column back to the training set
df_train_final['TARGET'] = df_target

# Clean up any potential duplicate columns (often created after merge/concat operations)
df_train_final = df_train_final.loc[:,~df_train_final.columns.duplicated()]
df_test_final = df_test_final.loc[:,~df_test_final.columns.duplicated()]

print(f"✅ One-Hot Encoding completed.")


# --- 4. Final Verification and Preparation for Modeling ---
print(f"\nFinal Train set shape: {df_train_final.shape}")
print(f"Final Test set shape: {df_test_final.shape}")

print(f"Total number of features after encoding: {df_train_final.shape[1]}")

--- Starting Feature Encoding (Block 13) ---
Applying Label Encoding to 3 features.
✅ Label Encoding completed.
Applying One-Hot Encoding to 13 features.
✅ One-Hot Encoding completed.

Final Train set shape: (307511, 266)
Final Test set shape: (48744, 265)
Total number of features after encoding: 266
